In [77]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 581.9 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 1.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.1 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 22.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 25.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 8.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 24.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 23.3 MB

In [ ]:
import pandas as pd

import sqlite3
import pandas as pd


from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS



In [72]:
with sqlite3.connect('data/pocos-news.db') as conn:
    df = pd.read_sql_query(
        "SELECT * FROM ondapocos;",
        conn,
        parse_dates=["date"]
    )


df = df.sort_values("date", ascending=False)
df['content'] = df['content'].str.split('Receba as notícias através').str[0]

In [73]:
%%time

metadata = df.drop(columns=["id", "content"]).to_dict(orient="records")
content = df['content'].to_list()
documents = [Document(page_content=c, metadata=m) for c, m in zip(content, metadata)]

CPU times: user 503 ms, sys: 2.86 ms, total: 506 ms
Wall time: 505 ms


In [78]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="clips/mfaq")

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1.65k/1.65k [00:00<00:00, 3.19MB/s]
1_Pooling/config.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:00<00:00, 573kB/s]
README.md: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3.74k/3.74k [00:00<00:00, 7.58MB/s]
added_tokens.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48.0/48.0 [00:00<00:00, 109kB/s]
config_sentence_transformer

In [79]:
%%time

from langchain.vectorstores import FAISS

# -- demora ----------------------------
vector_store = FAISS.from_documents(documents, embeddings)
vector_store.save_local("data/vectordatabase.db")


CPU times: user 3h 44min 58s, sys: 21min 16s, total: 4h 6min 15s
Wall time: 32min 15s


# QDrant

In [2]:
!pip install qdrant-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 359.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 621.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 8.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 8.8 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [7]:
import pandas as pd

import sqlite3
import pandas as pd


from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

from langchain.schema import Document

In [40]:
with sqlite3.connect('data/pocos-news.db') as conn:
    df = pd.read_sql_query(
        # "SELECT * FROM ondapocos LIMIT 1000;",
        "SELECT * FROM ondapocos;",
        conn,
        parse_dates=["date"]
    )


df = df.sort_values("date", ascending=False)
df['content'] = df['content'].str.split('Receba as notícias através').str[0]

In [41]:
%%time

metadata = df.drop(columns=["id", "content"]).to_dict(orient="records")
content = df['content'].to_list()
documents = [Document(page_content=c, metadata=m) for c, m in zip(content, metadata)]

CPU times: user 22.8 ms, sys: 0 ns, total: 22.8 ms
Wall time: 22.5 ms


In [42]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="clips/mfaq")

/opt/conda/lib/python3.11/site-packages/transformers/configuration_utils.py:387: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [43]:
%%time

import os
import shutil



from langchain.vectorstores import Qdrant


db_path = "data/qdrant"

if os.path.exists(db_path):
    try:
        shutil.rmtree(db_path)
    except OSError as e:
        print("Error: %s - %s." % (e.filename, e.strerror))

qdrant = Qdrant.from_documents(
    documents,
    embeddings,
    path="data/qdrant",
    collection_name="noticias",
    force_recreate=True,
)

CPU times: user 19min 58s, sys: 1min 47s, total: 21min 45s
Wall time: 2min 52s


In [ ]:
qdrant = Qdrant.